In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Reshape, Dense, GRU, Bidirectional
from tensorflow.keras.models import Model
from google.colab import drive
import numpy as np
import os
import zipfile
import shutil
import cv2
import pickle

In [ ]:
!git clone https://github.com/AI-FREE-Team/Traditional-Chinese-Handwriting-Dataset.git

Cloning into 'Traditional-Chinese-Handwriting-Dataset'...
remote: Enumerating objects: 174, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 174 (delta 5), reused 1 (delta 1), pack-reused 167 (from 1)
Receiving objects: 100% (174/174), 77.40 MiB | 29.99 MiB/s, done.
Resolving deltas: 100% (78/78), done.


In [ ]:
OutputFolder = '/content/Handwritten_Data'

In [ ]:
import os

if not os.path.exists(OutputFolder):
  os.mkdir(OutputFolder)
  print( f'Create the new "{OutputFolder}" folder' )

os.chdir(OutputFolder)

### 檢查路徑
!pwd

Create the new "/content/Handwritten_Data" folder
/content/Handwritten_Data


In [ ]:
CompressedFiles = []

os.chdir('/content/Traditional-Chinese-Handwriting-Dataset/data')

for item in os.listdir():
  if item.endswith('.zip'): # Check for ".zip" extension.
    file_path = os.path.abspath(item) # Get full path of the compressed file.
    CompressedFiles.append(file_path)

for file in CompressedFiles:
  # Construct a ZipFile object with the filename, and then extract it.
  zip_ref = zipfile.ZipFile(file).extractall(OutputFolder)

  source_path = OutputFolder + '/cleaned_data(50_50)'
  img_list = os.listdir(source_path)

  for img in img_list:
      shutil.move(source_path + '/' + img, OutputFolder) # Move a file to another location.

  shutil.rmtree(OutputFolder + '/cleaned_data(50_50)')
  print(f'Decompress successfully {file} ......')

print( 'Moving images according to traditional Chinese characters......' )

ImageList = os.listdir(OutputFolder)
ImageList = [img for img in ImageList if len(img)>1]
WordList = list(set([w.split('_')[0] for w in ImageList]))

for w in WordList:
  try:
    os.chdir(OutputFolder) # Change the current working directory to OutputPath.
    os.mkdir(w) # Create the new word folder in OutputPath.
    MoveList = [img for img in ImageList if w in img]

  except:
    os.chdir(OutputFolder)
    MoveList = [ img for img in ImageList if w in img ]

  finally:
    for img in MoveList:
      old_path = OutputFolder + '/' + img
      new_path = OutputFolder + '/' + w + '/' + img
      shutil.move( old_path, new_path )

print( 'Data Deployment completed.' )

In [ ]:
a=0
b=0

for item in os.listdir(OutputFolder):
  a += 1
  for i in os.listdir(OutputFolder + '/' + item):
    b +=1


print('總共: ' + str(a) + ' 個字(資料夾) / 總共: ' + str(b) + '個樣本')
print('平均每個字有: ' + str(b/a) + ' 個樣本')

In [ ]:
drive.mount('/content/drive')

In [ ]:
# Path to your dataset
dataset_path = OutputFolder

# Extract all folder names (characters)
characters = sorted(os.listdir(dataset_path))  # Assumes folder names are characters

# Create a character-to-index mapping
char_to_idx = {char: idx for idx, char in enumerate(characters)}
idx_to_char = {idx: char for char, idx in char_to_idx.items()}

print("Character-to-Index Mapping:", char_to_idx)


In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling1D

def build_crnn(input_shape, num_classes):
    input_layer = Input(shape=input_shape, name="image_input")

    # Convolutional layers
    x = Conv2D(64, (3, 3), activation="relu", padding="same")(input_layer)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(128, (3, 3), activation="relu", padding="same")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Reshape for RNN layers
    shape = x.shape
    x = Reshape((shape[1], shape[2] * shape[3]))(x)

    # Bidirectional GRU layers
    x = Bidirectional(GRU(256, return_sequences=True))(x)
    x = Bidirectional(GRU(256, return_sequences=True))(x)

    # Pooling to collapse the sequence dimension
    x = GlobalAveragePooling1D()(x)

    # Output layer (dense layer with softmax activation)
    output_layer = Dense(num_classes, activation="softmax")(x)

    # Define the model
    model = Model(inputs=input_layer, outputs=output_layer)
    return model

In [ ]:
# Model Parameters
input_shape = (32, 128, 1)  # Height, Width, Channels (grayscale image)
num_classes = 4803  # Number of Chinese characters to recognize

In [ ]:
# Instantiate the model
model = build_crnn(input_shape, num_classes)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ image_input (InputLayer)             │ (None, 32, 128, 1)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 32, 128, 64)         │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 16, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 16, 64, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 8, 32, 128)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 8, 4096)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 8, 512)              │       6,687,744 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 8, 512)              │       1,182,720 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 512)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 4803)                │       2,463,939 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,408,899 (39.71 MB)

 Trainable params: 10,408,899 (39.71 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
def preprocess_data(image, labels, char_to_idx, image_size=(128, 32)):
  image = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
  image = cv2.resize(image, image_size)  # Resize to uniform size
  image = np.expand_dims(image, axis=-1)  # Add channel dimension
  # image = np.expand_dims(image, axis=0)  # Add channel dimension
  image = image.astype("float32") / 255.0  # Normalize to [0, 1]
  print(image.shape)
  return np.array(image)

In [ ]:
def load_image_dataset(dataset_path, char_to_idx, image_size=(128, 32)):
    images = []
    labels = []
    for char_folder in os.listdir(dataset_path):
        char_path = os.path.join(dataset_path, char_folder)
        if os.path.isdir(char_path):
            label_idx = char_to_idx[char_folder]
            index = 0
            for image_file in os.listdir(char_path):
                index = index + 1
                print(f"{char_folder} and {image_file}")
                image_path = os.path.join(char_path, image_file)
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                image = cv2.resize(image, image_size)
                image = image.astype("float32") / 255.0
                image = np.expand_dims(image, axis=-1)
                images.append(image)
                labels.append(label_idx)
                if index == 10:
                    break
    return np.array(images), np.array(labels)

images, labels = load_image_dataset(dataset_path, char_to_idx)

print("Images shape:", images.shape)
print("Labels shape:", labels.shape)


In [ ]:
# One-hot encode labels for categorical output
from tensorflow.keras.utils import to_categorical

num_classes = len(char_to_idx)
labels = labels-1
labels_one_hot = to_categorical(labels, num_classes=num_classes)

# Train the CRNN model
model.fit(images, labels_one_hot, batch_size=16, epochs=5, validation_split=0.5)


Epoch 1/5
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 45s 30ms/step - accuracy: 0.0438 - loss: 7.3419 - val_accuracy: 0.0000e+00 - val_loss: 12.2578
Epoch 2/5
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 80s 28ms/step - accuracy: 0.5123 - loss: 2.1490 - val_accuracy: 4.1641e-05 - val_loss: 13.8288
Epoch 3/5
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 81s 28ms/step - accuracy: 0.8256 - loss: 0.7990 - val_accuracy: 8.3281e-05 - val_loss: 15.0056
Epoch 4/5
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 82s 28ms/step - accuracy: 0.9447 - loss: 0.3147 - val_accuracy: 4.1641e-05 - val_loss: 15.6139
Epoch 5/5
1501/1501 ━━━━━━━━━━━━━━━━━━━━ 82s 28ms/step - accuracy: 0.9828 - loss: 0.1371 - val_accuracy: 1.2492e-04 - val_loss: 16.2443


In [ ]:
model.save("chinese_handwriting_recognition_crnn.keras")

In [ ]:
def predict_image(image):
    # input_image = preprocess_data(image,labels,char_to_idx)
    # predictions = model.predict(input_image).flatten()
    # result_images = idx_to_char.get(np.argmax(predictions))
    # return result_images

    input_image = preprocess_data(image,labels,char_to_idx)
    # Add batch dimension to the input image
    input_image = np.expand_dims(input_image, axis=0)
    predictions = model.predict(input_image).flatten()
    result_images = idx_to_char.get(np.argmax(predictions))
    return result_images


In [ ]:
predict_image("/content/Handwritten_Data/丁/丁_0.png")

In [30]:
import gradio as gr

# Create Gradio interface
interface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="filepath", label="Upload an image"),
    outputs=gr.Textbox(label="Predicted Character"),
    title="Chinese Handwriting Recognition",
    description="Upload a handwritten Chinese character image to see the prediction."
)

# Launch the app
interface.launch(debug=True, share = True)


IMPORTANT: You are using gradio version 3.43.0, however version 4.44.1 is available, please upgrade.
--------
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://7467782113cab54891.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7865 <> https://7467782113cab54891.gradio.live


In [ ]:
!pip install gradio==3.43.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 100.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 14.1
    Uninstalling websockets-14.1:
      Successfully uninstalled websockets-14.1
  Attempting uninstall: pillow
    Found existing installation: pillow 11.0.0
    Uninstalling pillow-11.0.0:
      Successfully uninstalled pillow-11.0.0
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      S